In [167]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import string
import random
import pdb
%matplotlib inline

from HMM_sonnet import HiddenMarkovModel
from HMM_sonnet_helper import (
    #text_to_wordcloud,
    #states_to_wordclouds,
    parse_observations,
    sample_sentence,
    write_naive_line,
    visualize_sparsities,
    animate_emission
)

# Load Shakespearean and Spenser sonnets, and rhyme dictionary.

In [168]:
def create_dict(keys, values):
    return dict(zip(keys, values))

In [169]:
load_data = open("data/shakespeare.txt",'r').readlines()
load_syllables = open("data/Syllable_dictionary.txt",'r').readlines()
#load_syllables = open("Syllable_dictionary.txt",'r').readlines()

In [170]:
shakespeare = []
for i in range(len(load_data)):
    if load_data[i] != "\n":
        shakespeare.append(load_data[i][:-1].split(' '))

In [171]:
processed_shakespeare = []
for i in range(len(shakespeare)):
    line = []
    for j in range(len(shakespeare[i])):
        if shakespeare[i][j] != '':
            word = ''
            punc = ''
            if shakespeare[i][j][0] in ",.;:!'()?":
                line.append(shakespeare[i][j][0])
                if shakespeare[i][j][-1] in ",.;:!'()?":
                    if shakespeare[i][j][-2] in ",.;:!’'()?":
                        line.append(shakespeare[i][j][1:-2])
                        line.append(shakespeare[i][j][-2])
                        line.append(shakespeare[i][j][-1])
                    else:
                        line.append(shakespeare[i][j][1:-1])
                        line.append(shakespeare[i][j][-1])
                else:
                    line.append(shakespeare[i][j][1:])
            elif shakespeare[i][j][-1] in ",.;:!'’()?" and shakespeare[i][j][0] not in ",.;:!'()?":
                if shakespeare[i][j][-2] in ",.;:!’'()?":
                    line.append(shakespeare[i][j][:-2])
                    line.append(shakespeare[i][j][-2])
                    line.append(shakespeare[i][j][-1])
                else:
                    line.append(shakespeare[i][j][:-1])
                    line.append(shakespeare[i][j][-1])

            else:
                    line.append(shakespeare[i][j])
    processed_shakespeare.append(line)

shakespeare_lines = []
for line in processed_shakespeare:
    if len(line) > 1:
        shakespeare_lines.append(line)

In [172]:
syllables_notend = []
syllables_end = []
for i in range(len(load_syllables)):
    if load_syllables[i] != "\n":
        line = load_syllables[i][:-1].split(' ')
        if len(line) == 3:
            if line[1][0] == 'E':
                syllables_end.append([line[0],int(line[1][1:])])
                syllables_notend.append([line[0],[int(line[2]),int(line[2])]])
            else:
                syllables_notend.append([line[0],[int(line[2]),int(line[2])]])
        else:
            syllables_notend.append([line[0],[int(line[1]),int(line[1])]])
#syllables_notend.append(["'",[0,0]])
syllables_end_keys = np.array(syllables_end)[:,0]
syllables_end_values = np.array(syllables_end)[:,1].astype(int)

In [173]:
syllables_end_dict = create_dict(syllables_end_keys,syllables_end_values)

In [174]:
syllables_notend_keys = []
syllables_notend_values = []
for i in range(len(syllables_notend)):
    syllables_notend_keys.append(syllables_notend[i][0])
    syllables_notend_values.append(syllables_notend[i][1])
syllables_notend_keys[-1] = "'"
syllables_notend_values[-1] = [0,0]
syllables_notend_keys = np.array(syllables_notend_keys)
syllables_notend_values = np.array(syllables_notend_values).astype(int)
syllables_notend_dict = create_dict(syllables_notend_keys,syllables_notend_values)

In [175]:
wordtonumber_dict = create_dict(syllables_notend_keys,range(len(syllables_notend_keys)))
numbertoword_dict = create_dict(range(len(syllables_notend_keys)),syllables_notend_keys)

In [176]:
shakespeare_lines_token = []
for i in range(len(shakespeare_lines)):
    line = []
    for word in shakespeare_lines[i]:
        line.append(wordtonumber_dict[word.lower()])
    shakespeare_lines_token.append(line)

In [177]:
[numbertoword_dict[i] for i in range(3200,3214)]

['yours',
 'youth',
 "youth's",
 'youthful',
 'zealous',
 ',',
 '.',
 ';',
 ':',
 '!',
 '(',
 ')',
 '?',
 "'"]

# Train Hidden Markov Models on each set.

In [358]:
import random

class HiddenMarkovModel:
    '''
    Class implementation of Hidden Markov Models.
    '''

    def __init__(self, A, O):
        '''
        Initializes an HMM. Assumes the following:
            - States and observations are integers starting from 0. 
            - There is a start state (see notes on A_start below). There
              is no integer associated with the start state, only
              probabilities in the vector A_start.
            - There is no end state. 

        Arguments:
            A:          Transition matrix with dimensions L x L.
                        The (i, j)^th element is the probability of
                        transitioning from state i to state j. Note that
                        this does not include the starting probabilities.

            O:          Observation matrix with dimensions L x D.
                        The (i, j)^th element is the probability of
                        emitting observation j given state i.

        Parameters:
            L:          Number of states.

            D:          Number of observations.
            
            A:          The transition matrix.
            
            O:          The observation matrix.
            
            A_start:    Starting transition probabilities. The i^th element
                        is the probability of transitioning from the start
                        state to state i. For simplicity, we assume that
                        this distribution is uniform.
        '''

        self.L = len(A)
        self.D = len(O[0])
        self.A = A
        self.O = O
        self.A_start = [1. / self.L for _ in range(self.L)]


    def viterbi(self, x):
        '''
        Uses the Viterbi algorithm to find the max probability state 
        sequence corresponding to a given input sequence.

        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.

        Returns:
            max_seq:    Output sequence corresponding to x with the highest
                        probability.
        '''

        M = len(x)      # Length of sequence.

        # The (i, j)^th elements of probs and seqs are the max probability
        # of the prefix of length i ending in state j and the prefix
        # that gives this probability, respectively.
        #
        # For instance, probs[1][0] is the probability of the prefix of
        # length 1 ending in state 0.
        probs = [[0. for _ in range(self.L)] for _ in range(M + 1)]
        seqs = [['' for _ in range(self.L)] for _ in range(M + 1)]

        # Calculate initial prefixes and probabilities.
        for curr in range(self.L):
            probs[1][curr] = self.A_start[curr] * self.O[curr][x[0]]
            seqs[1][curr] = str(curr)

        # Calculate best prefixes and probabilities throughout sequence.
        for t in range(2, M + 1):
            # Iterate over all possible current states.
            for curr in range(self.L):
                max_prob = float("-inf")
                max_prefix = ''

                # Iterate over all possible previous states to find one
                # that would maximize the probability of the current state.
                for prev in range(self.L):
                    curr_prob = probs[t - 1][prev] \
                                * self.A[prev][curr] \
                                * self.O[curr][x[t - 1]]

                    # Continually update max probability and prefix.
                    if curr_prob >= max_prob:
                        max_prob = curr_prob
                        max_prefix = seqs[t - 1][prev]

                # Store the max probability and prefix.
                probs[t][curr] = max_prob
                seqs[t][curr] = max_prefix + str(curr)

        # Find the index of the max probability of a sequence ending in x^M
        # and the corresponding output sequence.
        max_i = max(enumerate(probs[-1]), key=lambda x: x[1])[0]
        max_seq = seqs[-1][max_i]

        return max_seq


    def forward(self, x, normalize=False):
        '''
        Uses the forward algorithm to calculate the alpha probability
        vectors corresponding to a given input sequence.

        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.

            normalize:  Whether to normalize each set of alpha_j(i) vectors
                        at each i. This is useful to avoid underflow in
                        unsupervised learning.

        Returns:
            alphas:     Vector of alphas.

                        The (i, j)^th element of alphas is alpha_j(i),
                        i.e. the probability of observing prefix x^1:i
                        and state y^i = j.

                        e.g. alphas[1][0] corresponds to the probability
                        of observing x^1:1, i.e. the first observation,
                        given that y^1 = 0, i.e. the first state is 0.
        '''

        M = len(x)      # Length of sequence.
        alphas = [[0. for _ in range(self.L)] for _ in range(M + 1)]

        # Note that alpha_j(0) is already correct for all j's.
        # Calculate alpha_j(1) for all j's.
        for curr in range(self.L):
            alphas[1][curr] = self.A_start[curr] * self.O[curr][x[0]]

        # Calculate alphas throughout sequence.
        for t in range(1, M):
            # Iterate over all possible current states.
            for curr in range(self.L):
                prob = 0

                # Iterate over all possible previous states to accumulate
                # the probabilities of all paths from the start state to
                # the current state.
                for prev in range(self.L):
                    prob += alphas[t][prev] \
                            * self.A[prev][curr] \
                            * self.O[curr][x[t]]

                # Store the accumulated probability.
                alphas[t + 1][curr] = prob

            if normalize:
                norm = sum(alphas[t + 1])
                for curr in range(self.L):
                    alphas[t + 1][curr] /= norm

        return alphas


    def backward(self, x, normalize=False):
        '''
        Uses the backward algorithm to calculate the beta probability
        vectors corresponding to a given input sequence.

        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.

            normalize:  Whether to normalize each set of alpha_j(i) vectors
                        at each i. This is useful to avoid underflow in
                        unsupervised learning.

        Returns:
            betas:      Vector of betas.

                        The (i, j)^th element of betas is beta_j(i), i.e.
                        the probability of observing prefix x^(i+1):M and
                        state y^i = j.

                        e.g. betas[M][0] corresponds to the probability
                        of observing x^M+1:M, i.e. no observations,
                        given that y^M = 0, i.e. the last state is 0.
        '''

        M = len(x)      # Length of sequence.
        betas = [[0. for _ in range(self.L)] for _ in range(M + 1)]

        # Initialize initial betas.
        for curr in range(self.L):
            betas[-1][curr] = 1

        # Calculate betas throughout sequence.
        for t in range(-1, -M - 1, -1):
            # Iterate over all possible current states.
            for curr in range(self.L):
                prob = 0

                # Iterate over all possible next states to accumulate
                # the probabilities of all paths from the end state to
                # the current state.
                for nxt in range(self.L):
                    if t == -M:
                        prob += betas[t][nxt] \
                                * self.A_start[nxt] \
                                * self.O[nxt][x[t]]

                    else:
                        prob += betas[t][nxt] \
                                * self.A[curr][nxt] \
                                * self.O[nxt][x[t]]

                # Store the accumulated probability.
                betas[t - 1][curr] = prob

            if normalize:
                norm = sum(betas[t - 1])
                for curr in range(self.L):
                    betas[t - 1][curr] /= norm

        return betas


    def supervised_learning(self, X, Y):
        '''
        Trains the HMM using the Maximum Likelihood closed form solutions
        for the transition and observation matrices on a labeled
        datset (X, Y). Note that this method does not return anything, but
        instead updates the attributes of the HMM object.

        Arguments:
            X:          A dataset consisting of input sequences in the form
                        of lists of variable length, consisting of integers 
                        ranging from 0 to D - 1. In other words, a list of
                        lists.

            Y:          A dataset consisting of state sequences in the form
                        of lists of variable length, consisting of integers 
                        ranging from 0 to L - 1. In other words, a list of
                        lists.

                        Note that the elements in X line up with those in Y.
        '''

        # Calculate each element of A using the M-step formulas.
        for curr in range(self.L):
            for nxt in range(self.L):
                num = 0.
                den = 0.

                for i in range(len(X)):
                    x = X[i]
                    y = Y[i]
                    M = len(x)
        
                    num += len([1 for i in range(M - 1) \
                                if y[i] == curr and y[i + 1] == nxt])
                    den += len([1 for i in range(M - 1) if y[i] == curr])

                self.A[curr][nxt] = num / den

        # Calculate each element of O using the M-step formulas.
        for curr in range(self.L):
            for xt in range(self.D):
                num = 0.
                den = 0.

                for i in range(len(X)):
                    x = X[i]
                    y = Y[i]
                    M = len(x)
        
                    num += len([1 for i in range(M) \
                                if y[i] == curr and x[i] == xt])
                    den += len([1 for i in range(M) if y[i] == curr])

                self.O[curr][xt] = num / den


    def unsupervised_learning(self, X, N_iters):
        '''
        Trains the HMM using the Baum-Welch algorithm on an unlabeled
        datset X. Note that this method does not return anything, but
        instead updates the attributes of the HMM object.

        Arguments:
            X:          A dataset consisting of input sequences in the form
                        of lists of length M, consisting of integers ranging
                        from 0 to D - 1. In other words, a list of lists.

            N_iters:    The number of iterations to train on.
        '''

        # Note that a comment starting with 'E' refers to the fact that
        # the code under the comment is part of the E-step.

        # Similarly, a comment starting with 'M' refers to the fact that
        # the code under the comment is part of the M-step.

        for iteration in range(1, N_iters + 1):
            if iteration % 10 == 0:
                print("Iteration: " + str(iteration))

            # Numerator and denominator for the update terms of A and O.
            A_num = [[0. for i in range(self.L)] for j in range(self.L)]
            O_num = [[0. for i in range(self.D)] for j in range(self.L)]
            A_den = [0. for i in range(self.L)]
            O_den = [0. for i in range(self.L)]

            # For each input sequence:
            for x in X:
                M = len(x)
                # Compute the alpha and beta probability vectors.
                alphas = self.forward(x, normalize=True)
                betas = self.backward(x, normalize=True)

                # E: Update the expected observation probabilities for a
                # given (x, y).
                # The i^th index is P(y^t = i, x).
                for t in range(1, M + 1):
                    P_curr = [0. for _ in range(self.L)]
                    
                    for curr in range(self.L):
                        P_curr[curr] = alphas[t][curr] * betas[t][curr]

                    # Normalize the probabilities.
                    norm = sum(P_curr)
                    for curr in range(len(P_curr)):
                        P_curr[curr] /= norm

                    for curr in range(self.L):
                        if t != M:
                            A_den[curr] += P_curr[curr]
                        O_den[curr] += P_curr[curr]
                        O_num[curr][x[t - 1]] += P_curr[curr]

                # E: Update the expectedP(y^j = a, y^j+1 = b, x) for given (x, y)
                for t in range(1, M):
                    P_curr_nxt = [[0. for _ in range(self.L)] for _ in range(self.L)]

                    for curr in range(self.L):
                        for nxt in range(self.L):
                            P_curr_nxt[curr][nxt] = alphas[t][curr] \
                                                    * self.A[curr][nxt] \
                                                    * self.O[nxt][x[t]] \
                                                    * betas[t + 1][nxt]

                    # Normalize:
                    norm = 0
                    for lst in P_curr_nxt:
                        norm += sum(lst)
                    for curr in range(self.L):
                        for nxt in range(self.L):
                            P_curr_nxt[curr][nxt] /= norm

                    # Update A_num
                    for curr in range(self.L):
                        for nxt in range(self.L):
                            A_num[curr][nxt] += P_curr_nxt[curr][nxt]

            for curr in range(self.L):
                for nxt in range(self.L):
                    self.A[curr][nxt] = A_num[curr][nxt] / A_den[curr]

            for curr in range(self.L):
                for xt in range(self.D):
                    self.O[curr][xt] = O_num[curr][xt] / O_den[curr]

    def generate_emission(self, M):
        '''
        Generates an emission of length M, assuming that the starting state
        is chosen uniformly at random. 

        Arguments:
            M:          Length of the emission to generate.

        Returns:
            emission:   The randomly generated emission as a list.

            states:     The randomly generated states as a list.
        '''

        emission = []
        state = random.choice(range(self.L))
        states = []

        for t in range(M):
            # Append state.
            states.append(state)

            # Sample next observation.
            rand_var = random.uniform(0, 1)
            next_obs = 0

            while rand_var > 0:
                rand_var -= self.O[state][next_obs]
                next_obs += 1

            next_obs -= 1
            emission.append(next_obs)

            # Sample next state.
            rand_var = random.uniform(0, 1)
            next_state = 0

            while rand_var > 0:
                rand_var -= self.A[state][next_state]
                next_state += 1

            next_state -= 1
            state = next_state

        return emission, states
    
    def naive_line(self, syl_dic, end_dic, number_to_word): #Think of how to add syllable dictionary.
        '''
        Generates a sonnet line of ten syllables, without rhyme or meter
        Arguments:
            M:          Length of the emission to generate.
        Returns:
            emission:   The randomly generated emission as a list.
            states:     The randomly generated states as a list.
        '''
        emission = []
        state = random.choice(range(self.L))
        states = []
        n_syllables = 0

        while n_syllables < 10:
            # Append state.
            states.append(state)
            
            #IMPORTANT: create a 'try' structure for n_syllables.
            
            # Sample next observation.
            rand_var = random.uniform(0, 1)
            next_obs = 0 

            while rand_var > 0:
                rand_var -= self.O[state][next_obs]
                next_obs += 1

            next_obs -= 1                 
            
            #Obtain number of syllables for this word choice.
            word_next_obs = number_to_word[next_obs]
            new_syllable = random.choice(syl_dic[word_next_obs])
            end_syllable = 0
            
            #Check if selected word is an end-syllable case.
            if next_obs in end_dic.keys():
                end_syllable = random.choice(end_dic[word_next_obs])
            
            if n_syllables + new_syllable > 10 or n_syllables + end_syllable > 10:
                pass
            else:
                emission.append(next_obs)

                # Sample next state.
                rand_var = random.uniform(0, 1)
                next_state = 0

                while rand_var > 0:
                    rand_var -= self.A[state][next_state]
                    next_state += 1
            
                #Create end condition, n_syllables == 10.
                if n_syllables + new_syllable == 10 or n_syllables + end_syllable == 10:
                    pass
                else:
                    next_state -= 1
                    state = next_state
                
                n_syllables += new_syllable
        return emission, states
        
    def haiku_line(self, syl_dic, end_dic, number_to_word,num_syllables): #Think of how to add syllable dictionary.
        '''
        Generates a sonnet line of ten syllables, without rhyme or meter
        Arguments:
            M:          Length of the emission to generate.
        Returns:
            emission:   The randomly generated emission as a list.
            states:     The randomly generated states as a list.
        '''
        emission = []
        state = random.choice(range(self.L))
        states = []
        n_syllables = 0

        while n_syllables < num_syllables:
            # Append state.
            states.append(state)
            
            #IMPORTANT: create a 'try' structure for n_syllables.
            
            # Sample next observation.
            rand_var = random.uniform(0, 1)
            next_obs = 0 

            while rand_var > 0:
                rand_var -= self.O[state][next_obs]
                next_obs += 1

            next_obs -= 1                 
            
            #Obtain number of syllables for this word choice.
            word_next_obs = number_to_word[next_obs]
            new_syllable = random.choice(syl_dic[word_next_obs])
            end_syllable = 0
            
            #Check if selected word is an end-syllable case.
            if next_obs in end_dic.keys():
                end_syllable = random.choice(end_dic[word_next_obs])
            
            if n_syllables + new_syllable > num_syllables or n_syllables + end_syllable > num_syllables:
                pass
            else:
                emission.append(next_obs)

                # Sample next state.
                rand_var = random.uniform(0, 1)
                next_state = 0

                while rand_var > 0:
                    rand_var -= self.A[state][next_state]
                    next_state += 1
            
                #Create end condition, n_syllables == 10.
                if n_syllables + new_syllable == num_syllables or n_syllables + end_syllable == num_syllables:
                    pass
                else:
                    next_state -= 1
                    state = next_state
                
                n_syllables += new_syllable
        
        
        '''
        #Treat edge case of end syllables.
        if emission[-1] in end_dic.keys(): #FIX FIX FIX
            #FIX: Missing some while loop to keep trying words.
            
            n_syllables -= new_syllable
            emission = emission[:-1]
            
            while n_syllables < 10: #Do we need this?
            
                #Draw an end word.
                rand_var = random.uniform(0, 1)
                next_obs = 0

                while rand_var > 0:
                    rand_var -= self.O[state][next_obs]
                    next_obs += 1

                next_obs -= 1   
            
                #Check whether this observation has an end-specific syllable count.
                if next_obs in end_dic: #FIX FIX FIX
                    if n_syllables + end_dic[next_obs] != 10:
                        pass
                    else:
                        emission.append(next_obs)

                        # Sample next state.
                        rand_var = random.uniform(0, 1)
                        next_state = 0

                        while rand_var > 0:
                            rand_var -= self.A[state][next_state]
                            next_state += 1
            
                        #Create end condition, n_syllables == 10. DO WE NEED THIS?
                        if n_syllables + end_dic[next_obs] == 10:
                            pass
                        else:
                            next_state -= 1
                            state = next_state
                        
                
                        n_syllables += end_dic[next_obs]
                else: #Case where the word is not in the end-syllable dictionary.
                    if n_syllables + syl_dic[next_obs] != 10:
                        pass
                    else:
                        emission.append(next_obs)

                        # Sample next state.
                        rand_var = random.uniform(0, 1)
                        next_state = 0

                        while rand_var > 0:
                            rand_var -= self.A[state][next_state]
                            next_state += 1
            
                        #Create end condition, n_syllables == 10. DO WE NEED THIS?
                        if n_syllables + syl_dic[next_obs] == 10:
                            pass
                        else:
                            next_state -= 1
                            state = next_state
                        
                
                        n_syllables += syl_dic[next_obs]
        
        '''
        
        return emission, states

    def rhyme_line(self, syl_dic, end_dic, number_to_word, word_to_number, seed): #Think of how to add syllable dictionary.
        '''
        Generates a sonnet line of ten syllables, without rhyme or meter
        Arguments:
            M:          Length of the emission to generate.
        Returns:
            emission:   The randomly generated emission as a list.
            states:     The randomly generated states as a list.
        '''
        emission = [seed]
        state = np.random.choice(range(self.L), p = np.array(hmm_shakespeare_4.O)[:,word_to_number[seed]]/np.array(hmm_shakespeare_4.O)[:,word_to_number[seed]].sum()) #assuming uniform prior
        states = []
        states.append(state)
        rand_var = random.uniform(0, 1)
        next_state = 0
        while rand_var > 0:
            rand_var -= self.A[next_state][state]/np.array(self.A)[next_state,state].sum()
            next_state += 1
        if seed in end_dic.keys():
            n_syllables = end_dic[seed]
        else:
            n_syllables = random.choice(syl_dic[seed])

        while n_syllables < 10:
            # Append state.
            states.insert(0,state)
            
            #IMPORTANT: create a 'try' structure for n_syllables.
            
            # Sample next observation.
            rand_var = random.uniform(0, 1)
            next_obs = 0 

            while rand_var > 0:
                rand_var -= self.O[state][next_obs]
                next_obs += 1

            next_obs -= 1                 
            
            #Obtain number of syllables for this word choice.
            word_next_obs = number_to_word[next_obs]
            new_syllable = random.choice(syl_dic[word_next_obs])
            #end_syllable = 0
            
            #Check if selected word is an end-syllable case.
            #if next_obs in end_dic.keys():
            #    end_syllable = random.choice(end_dic[word_next_obs])
            
            if n_syllables + new_syllable > 10:
                pass
            else:
                emission.insert(0,number_to_word[next_obs])

                # Sample next state.
                rand_var = random.uniform(0, np.array(self.A)[:,state].sum())
                next_state = 0

                while rand_var > 0:
                    rand_var -= self.A[next_state][state]
                    next_state += 1
            
                #Create end condition, n_syllables == 10.
                if n_syllables + new_syllable == 10:
                    pass
                else:
                    next_state -= 1
                    state = next_state
                
                n_syllables += new_syllable
        return emission, states
    
    def probability_alphas(self, x):
        '''
        Finds the maximum probability of a given input sequence using
        the forward algorithm.

        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.

        Returns:
            prob:       Total probability that x can occur.
        '''

        # Calculate alpha vectors.
        alphas = self.forward(x)

        # alpha_j(M) gives the probability that the output sequence ends
        # in j. Summing this value over all possible states j gives the
        # total probability of x paired with any output sequence, i.e. the
        # probability of x.
        prob = sum(alphas[-1])
        return prob


    def probability_betas(self, x):
        '''
        Finds the maximum probability of a given input sequence using
        the backward algorithm.

        Arguments:
            x:          Input sequence in the form of a list of length M,
                        consisting of integers ranging from 0 to D - 1.

        Returns:
            prob:       Total probability that x can occur.
        '''

        betas = self.backward(x)

        # beta_j(0) gives the probability of the output sequence. Summing
        # this over all states and then normalizing gives the total
        # probability of x paired with any output sequence, i.e. the
        # probability of x.
        prob = sum([betas[1][k] * self.A_start[k] * self.O[k][x[0]] \
            for k in range(self.L)])

        return prob


def supervised_HMM(X, Y):
    '''
    Helper function to train a supervised HMM. The function determines the
    number of unique states and observations in the given data, initializes
    the transition and observation matrices, creates the HMM, and then runs
    the training function for supervised learning.

    Arguments:
        X:          A dataset consisting of input sequences in the form
                    of lists of variable length, consisting of integers 
                    ranging from 0 to D - 1. In other words, a list of lists.

        Y:          A dataset consisting of state sequences in the form
                    of lists of variable length, consisting of integers 
                    ranging from 0 to L - 1. In other words, a list of lists.
                    Note that the elements in X line up with those in Y.
    '''

    # Make a set of observations.
    observations = set()
    for x in X:
        observations |= set(x)

    # Make a set of states.
    states = set()
    for y in Y:
        states |= set(y)
    
    # Compute L and D.
    L = len(states)
    D = len(observations)

    # Randomly initialize and normalize matrices A and O.
    A = [[random.random() for i in range(L)] for j in range(L)]

    for i in range(len(A)):
        norm = sum(A[i])
        for j in range(len(A[i])):
            A[i][j] /= norm
    
    O = [[random.random() for i in range(D)] for j in range(L)]

    for i in range(len(O)):
        norm = sum(O[i])
        for j in range(len(O[i])):
            O[i][j] /= norm

    # Train an HMM with labeled data.
    HMM = HiddenMarkovModel(A, O)
    HMM.supervised_learning(X, Y)

    return HMM


def unsupervised_HMM(X, n_states, N_iters):
    '''
    Helper function to train an unsupervised HMM. The function determines the
    number of unique observations in the given data, initializes
    the transition and observation matrices, creates the HMM, and then runs
    the training function for unsupervised learing.

    Arguments:
        X:          A dataset consisting of input sequences in the form
                    of lists of variable length, consisting of integers 
                    ranging from 0 to D - 1. In other words, a list of lists.

        n_states:   Number of hidden states to use in training.
        
        N_iters:    The number of iterations to train on.
    '''

    # Make a set of observations.
    observations = set()
    for x in X:
        observations |= set(x)
    
    # Compute L and D.
    L = n_states
    D = len(observations)

    # Randomly initialize and normalize matrices A and O.
    A = [[random.random() for i in range(L)] for j in range(L)]

    for i in range(len(A)):
        norm = sum(A[i])
        for j in range(len(A[i])):
            A[i][j] /= norm
    
    # Randomly initialize and normalize matrix O.
    O = [[random.random() for i in range(D)] for j in range(L)]

    for i in range(len(O)):
        norm = sum(O[i])
        for j in range(len(O[i])):
            O[i][j] /= norm

    # Train an HMM with unlabeled data.
    HMM = HiddenMarkovModel(A, O)
    HMM.unsupervised_learning(X, N_iters)

    return HMM

In [301]:
def supervised_HMM(X, Y):
    '''
    Helper function to train a supervised HMM. The function determines the
    number of unique states and observations in the given data, initializes
    the transition and observation matrices, creates the HMM, and then runs
    the training function for supervised learning.

    Arguments:
        X:          A dataset consisting of input sequences in the form
                    of lists of variable length, consisting of integers 
                    ranging from 0 to D - 1. In other words, a list of lists.

        Y:          A dataset consisting of state sequences in the form
                    of lists of variable length, consisting of integers 
                    ranging from 0 to L - 1. In other words, a list of lists.
                    Note that the elements in X line up with those in Y.
    '''

    # Make a set of observations.
    observations = set()
    for x in X:
        observations |= set(x)
        
    # Make a set of states.
    states = set()
    for y in Y:
        states |= set(y)
    # Compute L and D.
    L = len(states)
    D = len(observations)

    # Randomly initialize and normalize matrices A and O.
    A = [[random.random() for i in range(L)] for j in range(L)]

    for i in range(len(A)):
        norm = sum(A[i])
        for j in range(len(A[i])):
            A[i][j] /= norm
    
    O = [[random.random() for i in range(D)] for j in range(L)]

    for i in range(len(O)):
        norm = sum(O[i])
        for j in range(len(O[i])):
            O[i][j] /= norm

    # Train an HMM with labeled data.
    HMM = HiddenMarkovModel(A, O)
    HMM.supervised_learning(X, Y)

    return HMM

def unsupervised_HMM(X, n_states, N_iters):
    '''
    Helper function to train an unsupervised HMM. The function determines the
    number of unique observations in the given data, initializes
    the transition and observation matrices, creates the HMM, and then runs
    the training function for unsupervised learing.

    Arguments:
        X:          A dataset consisting of input sequences in the form
                    of lists of variable length, consisting of integers 
                    ranging from 0 to D - 1. In other words, a list of lists.

        n_states:   Number of hidden states to use in training.
        
        N_iters:    The number of iterations to train on.
    '''
    # Make a set of observations.
    observations = set()
    for x in X:
        observations |= set(x)

    # Compute L and D.
    L = n_states
    D = len(observations)

    # Randomly initialize and normalize matrices A and O.
    A = [[random.random() for i in range(L)] for j in range(L)]

    for i in range(len(A)):
        norm = sum(A[i])
        for j in range(len(A[i])):
            A[i][j] /= norm
    
    # Randomly initialize and normalize matrix O.
    O = [[random.random() for i in range(D)] for j in range(L)]

    for i in range(len(O)):
        norm = sum(O[i])
        for j in range(len(O[i])):
            O[i][j] /= norm
    # Train an HMM with unlabeled data.
    HMM = HiddenMarkovModel(A, O)
    HMM.unsupervised_learning(X, N_iters)
    return HMM

In [302]:
########################################
# CS/CNS/EE 155 2018
# Problem Set 6
#
# Author:       Andrew Kang
# Description:  Set 6 HMM helper
########################################

import re
import numpy as np
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
from matplotlib import animation
from matplotlib.animation import FuncAnimation


####################
# WORDCLOUD FUNCTIONS
####################

def mask():
    # Parameters.
    r = 128
    d = 2 * r + 1

    # Get points in a circle.
    y, x = np.ogrid[-r:d-r, -r:d-r]
    circle = (x**2 + y**2 <= r**2)

    # Create mask.
    mask = 255 * np.ones((d, d), dtype=np.uint8)
    mask[circle] = 0

    return mask

def text_to_wordcloud(text, max_words=50, title='', show=True):
    plt.close('all')

    # Generate a wordcloud image.
    wordcloud = WordCloud(random_state=0,
                          max_words=max_words,
                          background_color='white',
                          mask=mask()).generate(text)

    # Show the image.
    if show:
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(title, fontsize=24)
        plt.show()

    return wordcloud

def states_to_wordclouds(hmm, obs_map, max_words=50, show=True):
    # Initialize.
    M = 100000
    n_states = len(hmm.A)
    obs_map_r = obs_map_reverser(obs_map)
    wordclouds = []

    # Generate a large emission.
    emission, states = hmm.generate_emission(M)

    # For each state, get a list of observations that have been emitted
    # from that state.
    obs_count = []
    for i in range(n_states):
        obs_lst = np.array(emission)[np.where(np.array(states) == i)[0]]
        obs_count.append(obs_lst)

    # For each state, convert it into a wordcloud.
    for i in range(n_states):
        obs_lst = obs_count[i]
        sentence = [obs_map_r[j] for j in obs_lst]
        sentence_str = ' '.join(sentence)

        wordclouds.append(text_to_wordcloud(sentence_str, max_words=max_words, title='State %d' % i, show=show))

    return wordclouds


####################
# HMM FUNCTIONS
####################

def parse_observations(text):
    # Convert text to dataset.
    lines = [line.split() for line in text.split('\n') if line.split()]

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []
        
        for word in line:
            word = re.sub(r'[^\w]', '', word).lower()
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map

def obs_map_reverser(obs_map):
    obs_map_r = {}

    for key in obs_map:
        obs_map_r[obs_map[key]] = key

    return obs_map_r

def sample_sentence(hmm, obs_map, n_words=100):
    # Get reverse map.
    obs_map_r = obs_map_reverser(obs_map)

    # Sample and convert sentence.
    emission, states = hmm.generate_emission(n_words)
    sentence = [obs_map_r[i] for i in emission]

    return ' '.join(sentence).capitalize() + '...'

def write_naive_line(hmm, syl_dic, end_dic, number_to_word, word_to_number):
    #Naively generate lines, without rhyme or meter.
    obs_map_r = word_to_number
    # Sample and convert sentence.
    emission, states = hmm.naive_line(syl_dic, end_dic, number_to_word) 
    line = [number_to_word[i] for i in emission]
    
    return ' '.join(line).capitalize(), states

def write_haiku_line(hmm, syl_dic, end_dic, number_to_word, word_to_number, num_syllables):
    #Naively generate lines, without rhyme or meter.
    obs_map_r = word_to_number
    # Sample and convert sentence.
    emission, states = hmm.haiku_line(syl_dic, end_dic, number_to_word,num_syllables) 
    line = [number_to_word[i] for i in emission]
    return ' '.join(line).capitalize(), states

####################
# HMM VISUALIZATION FUNCTIONS
####################

def visualize_sparsities(hmm, O_max_cols=50, O_vmax=0.1):
    plt.close('all')
    plt.set_cmap('viridis')

    # Visualize sparsity of A.
    plt.imshow(hmm.A, vmax=1.0)
    plt.colorbar()
    plt.title('Sparsity of A matrix')
    plt.show()

    # Visualize parsity of O.
    plt.imshow(np.array(hmm.O)[:, :O_max_cols], vmax=O_vmax, aspect='auto')
    plt.colorbar()
    plt.title('Sparsity of O matrix')
    plt.show()


####################
# HMM ANIMATION FUNCTIONS
####################

def animate_emission(hmm, obs_map, M=8, height=12, width=12, delay=1):
    # Parameters.
    lim = 1200
    text_x_offset = 40
    text_y_offset = 80
    x_offset = 580
    y_offset = 520
    R = 420
    r = 100
    arrow_size = 20
    arrow_p1 = 0.03
    arrow_p2 = 0.02
    arrow_p3 = 0.06
    
    # Initialize.
    n_states = len(hmm.A)
    obs_map_r = obs_map_reverser(obs_map)
    wordclouds = states_to_wordclouds(hmm, obs_map, max_words=20, show=False)

    # Initialize plot.    
    fig, ax = plt.subplots()
    fig.set_figheight(height)
    fig.set_figwidth(width)
    ax.grid('off')
    plt.axis('off')
    ax.set_xlim([0, lim])
    ax.set_ylim([0, lim])

    # Plot each wordcloud.
    for i, wordcloud in enumerate(wordclouds):
        x = x_offset + int(R * np.cos(np.pi * 2 * i / n_states))
        y = y_offset + int(R * np.sin(np.pi * 2 * i / n_states))
        ax.imshow(wordcloud.to_array(), extent=(x - r, x + r, y - r, y + r), aspect='auto', zorder=-1)

    # Initialize text.
    text = ax.text(text_x_offset, lim - text_y_offset, '', fontsize=24)
        
    # Make the arrows.
    zorder_mult = n_states ** 2 * 100
    arrows = []
    for i in range(n_states):
        row = []
        for j in range(n_states):
            # Arrow coordinates.
            x_i = x_offset + R * np.cos(np.pi * 2 * i / n_states)
            y_i = y_offset + R * np.sin(np.pi * 2 * i / n_states)
            x_j = x_offset + R * np.cos(np.pi * 2 * j / n_states)
            y_j = y_offset + R * np.sin(np.pi * 2 * j / n_states)
            
            dx = x_j - x_i
            dy = y_j - y_i
            d = np.sqrt(dx**2 + dy**2)

            if i != j:
                arrow = ax.arrow(x_i + (r/d + arrow_p1) * dx + arrow_p2 * dy,
                                 y_i + (r/d + arrow_p1) * dy + arrow_p2 * dx,
                                 (1 - 2 * r/d - arrow_p3) * dx,
                                 (1 - 2 * r/d - arrow_p3) * dy,
                                 color=(1 - hmm.A[i][j], ) * 3,
                                 head_width=arrow_size, head_length=arrow_size,
                                 zorder=int(hmm.A[i][j] * zorder_mult))
            else:
                arrow = ax.arrow(x_i, y_i, 0, 0,
                                 color=(1 - hmm.A[i][j], ) * 3,
                                 head_width=arrow_size, head_length=arrow_size,
                                 zorder=int(hmm.A[i][j] * zorder_mult))

            row.append(arrow)
        arrows.append(row)

    emission, states = hmm.generate_emission(M)

    def animate(i):
        if i >= delay:
            i -= delay

            if i == 0:
                arrows[states[0]][states[0]].set_color('red')
            elif i == 1:
                arrows[states[0]][states[0]].set_color((1 - hmm.A[states[0]][states[0]], ) * 3)
                arrows[states[i - 1]][states[i]].set_color('red')
            else:
                arrows[states[i - 2]][states[i - 1]].set_color((1 - hmm.A[states[i - 2]][states[i - 1]], ) * 3)
                arrows[states[i - 1]][states[i]].set_color('red')

            # Set text.
            text.set_text(' '.join([obs_map_r[e] for e in emission][:i+1]).capitalize())

            return arrows + [text]

    # Animate!
    print('\nAnimating...')
    anim = FuncAnimation(fig, animate, frames=M+delay, interval=1000)

    return anim

In [380]:
hmm_shakespeare_4 = unsupervised_HMM(shakespeare_lines_token, 16, 100)
#hmm_spenser_32 = unsupervised_HMM(spenser, 32, 100)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100


# Generate 14-line sonnets. Currently pseudocode.

In [294]:
punctuation = [",",".",";",":","!","'","’","(",")","?"," "]
punctuation_probs = np.zeros(len(",.;:!'’()?")+1) #,.;:!'’()?none
for i in range(len(shakespeare_lines)):
    last_element = shakespeare_lines[i][-1]
    if last_element == ",":
        punctuation_probs[0] += 1
    elif last_element == ".":
        punctuation_probs[1] += 1
    elif last_element == ";":
        punctuation_probs[2] += 1
    elif last_element == ":":
        punctuation_probs[3] += 1
    elif last_element == "!":
        punctuation_probs[4] += 1
    elif last_element == "'":
        punctuation_probs[5] += 1
    elif last_element == "’":
        punctuation_probs[6] += 1
    elif last_element == "(":
        punctuation_probs[7] += 1
    elif last_element == ")":
        punctuation_probs[8] += 1
    elif last_element == "?":
        punctuation_probs[9] += 1
    else:
        punctuation_probs[10] += 1
punctuation_probs = punctuation_probs/float(punctuation_probs.sum())

In [204]:
def Shakespeare_naive(hmm): #Takes trained HMM as input.
    #No rhyme here, only constraining number of syllables and lines.
    sonnet = ''
    hidden_arr = []
    for i in np.arange(14):
        line, h_states = write_naive_line(hmm, syllables_notend_dict, syllables_end_dict, numbertoword_dict, wordtonumber_dict)
        sonnet += line + np.random.choice(punctuation, p=punctuation_probs) + ' \n ' #Waiting on Chris for punctuation.
        hidden_arr.append(h_states)
    return sonnet, hidden_arr

def Shakespeare_haiku(hmm): #Takes trained HMM as input.
    #No rhyme here, only constraining number of syllables and lines.
    sonnet = ''
    hidden_arr = []
    for i in [5,7,5]:
        line, h_states = write_haiku_line(hmm, syllables_notend_dict, syllables_end_dict, numbertoword_dict, wordtonumber_dict,i)
        sonnet += line + ' \n ' #Waiting on Chris for punctuation.
        hidden_arr.append(h_states)
    return sonnet, hidden_arr

In [286]:
badone_begin = 98*14 
badone_end = badone_begin + 14
badtwo_begin = badone_end + 14*(125-99) + 1
badtwo_end = badtwo_begin + 11
print(shakespeare_lines[badone_begin])
print(shakespeare_lines[badone_end])
print(shakespeare_lines[badtwo_begin])
print(shakespeare_lines[badtwo_end])
#print(badone_begin, badone_end, badtwo_begin, badtwo_end)
print(np.arange(badone_begin, badone_end + 1))
print(np.arange(badtwo_begin, badtwo_end + 1))

['The', 'forward', 'violet', 'thus', 'did', 'I', 'chide', ',']
['But', 'sweet', ',', 'or', 'colour', 'it', 'had', "stol'n", 'from', 'thee', '.']
['O', 'thou', 'my', 'lovely', 'boy', 'who', 'in', 'thy', 'power', ',']
['And', 'her', 'quietus', 'is', 'to', 'render', 'thee', '.']
[1372 1373 1374 1375 1376 1377 1378 1379 1380 1381 1382 1383 1384 1385 1386]
[1751 1752 1753 1754 1755 1756 1757 1758 1759 1760 1761 1762]


In [287]:
num_sonnets = 154
usable_lines = [] #Remove the 12-line and 15-line sonnets, 99 and 126.
for i in np.arange(len(shakespeare_lines)):
    if i in np.arange(badone_begin, badone_end + 1):
        pass
    elif i in np.arange(badtwo_begin, badtwo_end + 1):
        pass
    else:
        usable_lines.append(i)  

num_lines = len(usable_lines)
num_sonnets = num_lines/14 #Should be 152, 154 minus the two oddballs.
   
rhyming_keys = []
punctuations = range(3205,3214)
    
for i in np.arange(num_sonnets):
    
    #Don't include punctuation
    for j in np.arange(3):
        
        #word1 = shakespeare_lines_token[usable_lines[int(14*i + 4*j)]][-1]
        if shakespeare_lines_token[usable_lines[int(14*i + 4*j)]][-1] in punctuations:
            if shakespeare_lines_token[usable_lines[int(14*i + 4*j)]][-2] in punctuations:
                word1 = shakespeare_lines_token[usable_lines[int(14*i + 4*j)]][-3]
            else:
                word1 = shakespeare_lines_token[usable_lines[int(14*i + 4*j)]][-2]
        else:
            word1 = shakespeare_lines_token[usable_lines[int(14*i + 4*j)]][-1]   
        
        if shakespeare_lines_token[usable_lines[int(14*i + 2 + 4*j)]][-1] in punctuations:
            if shakespeare_lines_token[usable_lines[int(14*i + 2 + 4*j)]][-2] in punctuations:
                word2 = shakespeare_lines_token[usable_lines[int(14*i + 2 + 4*j)]][-3]
            else:
                word2 = shakespeare_lines_token[usable_lines[int(14*i + 2 + 4*j)]][-2]
        else:
            word2 = shakespeare_lines_token[usable_lines[int(14*i + 2 + 4*j)]][-1]   
            
        rhyming_keys.append([word1, word2])

        
        if shakespeare_lines_token[usable_lines[int(14*i + 1 + 4*j)]][-1] in punctuations:
            if shakespeare_lines_token[usable_lines[int(14*i + 1 + 4*j)]][-2] in punctuations:
                word3 = shakespeare_lines_token[usable_lines[int(14*i + 1 + 4*j)]][-3]
            else:
                word3 = shakespeare_lines_token[usable_lines[int(14*i + 1 + 4*j)]][-2]
        else:
            word3 = shakespeare_lines_token[usable_lines[int(14*i + 1 + 4*j)]][-1]           
        
        if shakespeare_lines_token[usable_lines[int(14*i + 3 + 4*j)]][-1] in punctuations:
            if shakespeare_lines_token[usable_lines[int(14*i + 3 + 4*j)]][-2] in punctuations:
                word4 = shakespeare_lines_token[usable_lines[int(14*i + 3 + 4*j)]][-3]
            else:
                word4 = shakespeare_lines_token[usable_lines[int(14*i + 3 + 4*j)]][-2]
        else:
            word4 = shakespeare_lines_token[usable_lines[int(14*i + 3 + 4*j)]][-1]   
        
        rhyming_keys.append([word3, word4])    
            
            
    if shakespeare_lines_token[usable_lines[int(14*i) + 12]][-1] in punctuations:
        if shakespeare_lines_token[usable_lines[int(14*i + 12)]][-2] in punctuations:
            couplet1 = shakespeare_lines_token[usable_lines[int(14*i + 12)]][-3]
        else:
            couplet1 = shakespeare_lines_token[usable_lines[int(14*i + 12)]][-2]
    else:
        couplet1 = shakespeare_lines_token[usable_lines[int(14*i + 12)]][-1]  
        
    if shakespeare_lines_token[usable_lines[int(14*i) + 13]][-1] in punctuations:
        if shakespeare_lines_token[usable_lines[int(14*i + 13)]][-2] in punctuations:
            couplet2 = shakespeare_lines_token[usable_lines[int(14*i + 13)]][-3]
        else:
            couplet2 = shakespeare_lines_token[usable_lines[int(14*i + 13)]][-2]
    else:
        couplet2 = shakespeare_lines_token[usable_lines[int(14*i + 13)]][-1]    
        
    rhyming_keys.append([couplet1, couplet2])

rhyming_words = []
for i in np.arange(len(rhyming_keys)):
    rhyming_words.append([numbertoword_dict[rhyming_keys[i][0]], \
                          numbertoword_dict[rhyming_keys[i][1]]])

In [349]:
rhyming_words

[['increase', 'decease'],
 ['die', 'memory'],
 ['eyes', 'lies'],
 ['fuel', 'cruel'],
 ['ornament', 'content'],
 ['spring', 'niggarding'],
 ['be', 'thee'],
 ['brow', 'now'],
 ['field', 'held'],
 ['lies', 'eyes'],
 ['days', 'praise'],
 ['use', 'excuse'],
 ['mine', 'thine'],
 ['old', 'cold'],
 ['viewest', 'renewest'],
 ['another', 'mother'],
 ['womb', 'tomb'],
 ['husbandry', 'posterity'],
 ['thee', 'see'],
 ['prime', 'time'],
 ['be', 'thee'],
 ['spend', 'lend'],
 ['legacy', 'free'],
 ['abuse', 'use'],
 ['give', 'live'],
 ['alone', 'gone'],
 ['deceive', 'leave'],
 ['thee', 'be'],
 ['frame', 'same'],
 ['dwell', 'excel'],
 ['on', 'gone'],
 ['there', 'where'],
 ['left', 'bereft'],
 ['glass', 'was'],
 ['meet', 'sweet'],
 ['deface', 'place'],
 ['distilled', 'self-killed'],
 ['usury', 'thee'],
 ['loan', 'one'],
 ['art', 'depart'],
 ['thee', 'posterity'],
 ['fair', 'heir'],
 ['light', 'sight'],
 ['eye', 'majesty'],
 ['hill', 'still'],
 ['age', 'pilgrimage'],
 ['car', 'are'],
 ['day', 'way'],
 ['n

In [207]:
sonnet , h_states = Shakespeare_naive(hmm_shakespeare_4)
print(sonnet)
[print(h_states[i]) for i in range(14)]

When this therein coral grace thee knife your, 
 Golden stay doth left relief tempting and: 
 Are earth what serving well ' , . when but to and: 
 Sin too but comment , guess the offender's, 
 Needs or tend do night , thy asleep enjoys, 
 , you perceived but a love thousand . men's there: 
 Which delves as then no seen into which this: 
 Mourning much blamed defect brow thou and on, 
 Others to if those love all to age to, 
 Then , though love , be benefit he least did: 
 Twain of how spurring is spread were that in, 
 Check time , it due doth so world , and it hear, 
 , i'll ' , peep , seeing dulness to thou from can, 
 Self , thy see to truly giving ) o had, 
 
[0, 0, 1, 0, 1, 0, 1, 3]
[3, 2, 1, 3, 2, 1, 0]
[3, 2, 1, 3, 2, 2, 1, 1, 0, 1, 1, 0]
[2, 1, 3, 2, 1, 3, 2, 1]
[2, 3, 2, 3, 2, 1, 3, 2, 1]
[1, 3, 2, 1, 3, 2, 1, 1, 0, 1]
[2, 1, 0, 1, 3, 2, 1, 0, 1, 1]
[1, 3, 2, 2, 2, 1, 0, 1]
[3, 2, 1, 3, 2, 1, 3, 2, 1]
[0, 1, 3, 2, 1, 3, 2, 1, 0, 1, 1]
[2, 1, 1, 0, 1, 0, 1, 0, 1]
[0, 0, 1, 0, 0

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
haiku , haiku_states = Shakespeare_haiku(hmm_shakespeare_4)
print(haiku)
[print(haiku_states[i]) for i in range(3)]

In [360]:
sonnets, states = hmm_shakespeare_4.rhyme_line(syllables_notend_dict,syllables_end_dict,numbertoword_dict,wordtonumber_dict,"winged")
print(' '.join(sonnets))

looks his , so pays not equal far winged


In [385]:
def Shakespeare_rhyme(hmm):
    sonnet = ''
    for i in np.arange(3):
        seeds1 = random.choice(rhyming_words)
        seeds2 = random.choice(rhyming_words)
        seed1 = seeds1[0]
        seed2 = seeds2[0]
        seed3 = seeds1[1]
        seed4 = seeds2[1]
        sonnet += " ".join(hmm.rhyme_line(syllables_notend_dict,syllables_end_dict,numbertoword_dict,wordtonumber_dict,seed1)[0]).capitalize()+np.random.choice(punctuation, p=punctuation_probs) + '\n'
        sonnet += " ".join(hmm.rhyme_line(syllables_notend_dict,syllables_end_dict,numbertoword_dict,wordtonumber_dict,seed2)[0]).capitalize()+np.random.choice(punctuation, p=punctuation_probs) + '\n'
        sonnet += " ".join(hmm.rhyme_line(syllables_notend_dict,syllables_end_dict,numbertoword_dict,wordtonumber_dict,seed3)[0]).capitalize()+np.random.choice(punctuation, p=punctuation_probs) + '\n'
        sonnet += " ".join(hmm.rhyme_line(syllables_notend_dict,syllables_end_dict,numbertoword_dict,wordtonumber_dict,seed4)[0]).capitalize()+np.random.choice(punctuation, p=punctuation_probs) + '\n'
        sonnet += "\n"
    seeds3 = random.choice(rhyming_words)
    sonnet += " " + " ".join(hmm.rhyme_line(syllables_notend_dict,syllables_end_dict,numbertoword_dict,wordtonumber_dict,seeds3[0])[0]).capitalize()+np.random.choice(punctuation, p=punctuation_probs) + '\n '
    sonnet += " ".join(hmm.rhyme_line(syllables_notend_dict,syllables_end_dict,numbertoword_dict,wordtonumber_dict,seeds3[1])[0]).capitalize()+np.random.choice(punctuation, p=punctuation_probs) + '\n '
    return sonnet

In [412]:
print(Shakespeare_rhyme(hmm_shakespeare_4))

Traffic to still my added unrest kings,
Amis thou no harsh on an pleasure worth:
So now so night entombed , and some things.
Faults yet with remember in thou him , forth,

Nor seen of yet he but never which say:
Might but look garments thy seeing . distilled,
There shame more dear argument when up pay,
Doth art on thy strive , time's bankrupt self-killed,

And withering have from quality costs 
Even to extern name . or audit gain,
Anew have pass roses from the sweet boast,
Self from make woman pen mayst doom dost main,

 The beauty's need but i draw this youth shore,
 World i so it more of than this eyes store,
 


In [389]:
state = 0
next_state = 0
rand_var = random.uniform(0, np.array(hmm_shakespeare_4.A)[:,state].sum())
print(rand_var)
while rand_var > 0:
    rand_var -= hmm_shakespeare_4.A[next_state][state]
    print(rand_var)
    next_state += 1
#np.array(hmm_shakespeare_4.A)[next_state,state]

0.677768689299
0.677768689299
0.677768507937
0.677768507937
0.677768507746
0.677768507746
0.677768065212
0.677768065212
0.677767869573
0.677767869573
0.677767869573
0.677767869573
0.160872687937
0.0155489052722
0.0155489052677
-0.29464473592
